# FINAL TEST FOR PROJECT

In [ ]:
my_pwd = "*"
import pymongo # for MongoDB
import psycopg2 # for PostgresQL
import pandas as pd 
import re
import json
import time
import re
import datetime
from datetime import datetime
import datetime

### HELPER FUNCTION 1 

In [ ]:
def get_text_sorted(text):
    res = [re.sub(r'[^\w\s]', '', s) for s in text.lower().split()]
    res.sort()
    return res

def get_hashtags_sorted(hlist):
    hlist.sort()
    return hlist

def get_JSON_date(s):
    d=s.split()
    t=d[3].split(':')
    months={'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
    m=datetime.datetime(day=int(d[2]), month=months[d[1]], hour=int(t[0]), minute=int(t[1]), second=int(t[2]), microsecond=int(d[4]), year=int(d[5]))
    return m.isoformat()

### HELPER FUNCTION 2

In [ ]:
# Function for Inserting values to user_db and hashtag_db (PostgreSQL)

def insert_to_userdb(screen_name,user_id,followers_count,tweet_id):
    """ insert values into the user table """
    
    sql = """INSERT INTO user_db(screen_name, user_id ,followers_count, tweet_ids) VALUES(%s,%s,%s,%s);"""
    try:
        # execute the INSERT statement
        cur.execute("ROLLBACK")
        cur.execute(sql, (screen_name,user_id,followers_count,tweet_id))
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

def update_tweet_ids_userdb(screen_name_,tweet_id):
    """ update values into the userdb table """ 
    sql =   """ UPDATE user_db SET tweet_ids = array_append((SELECT tweet_ids FROM user_db WHERE screen_name=(%s)),(%s)) WHERE screen_name = (%s) """
    try:
        # execute the UPDATE statement
        cur.execute("ROLLBACK")
        cur.execute(sql, (screen_name_,tweet_id,screen_name_))
    except (Exception, psycopg2.DatabaseError) as error:
        print(error) 
    

def get_user_screen_names():
    cur.execute("""SELECT screen_name FROM user_db""")
    output = cur.fetchall()
    res = []
    for x in output:
        res.append(x[0])
    return res

def get_user_tweet_ids(sc_name):
    """ get tweet_ids values from corresponding user""" 
    sql =   """SELECT tweet_ids FROM user_db WHERE screen_name = %s"""
    try:
        # execute the UPDATE statement
        cur.execute("ROLLBACK")
        cur.execute(sql, (sc_name,))
        output = cur.fetchall()
        if len(output) !=0:
            return output[0][0]
    except (Exception, psycopg2.DatabaseError) as error:
        print(error) 

### SEARCH FUNCTIONS

In [ ]:
# 1. Search by string should be fixed using binary search logic based on the interim report
def binary_search(text_array, low, high, s):

    if high >= low:
        mid = (high + low) // 2
        if text_array[mid].lower() == s.lower():
            return True
        elif text_array[mid] > s:
            return binary_search(text_array, low, mid - 1, s)
        else:
            return binary_search(text_array, mid + 1, high, s)
    else:
        return False
 
def search_by_text():
    print('Enter a word you want to search:')
    s = input()
    print('Searching for ' + s + " ...")
    if type(s) != str:
        s = s.toString()
    ret=list()
    for doc in tweets.find().sort('interactions', -1):
        if binary_search(doc['text_sorted'], 0, len(doc['text_sorted'])-1, s):
            ret.append({'Tweet Text':doc['text'],
                        'Author of Tweet':doc['user_id'],
                        'When Created:':doc["created_at"],
                       'Retweet Count':doc['retweet_count']
                        })
    return ret


def search_by_ht():
    print('Enter a hashtag you want to search:')
    s = input()
    print('Searching for ' + s + " ...")
    if type(s) != str:
        s = s.toString()
    ret=list()
    for doc in tweets.find().sort('interactions', -1):
        if binary_search(doc['hashtags'], 0, len(doc['hashtags'])-1, s):
            ret.append({'Tweet Text':doc['text'],
                        'Author of Tweet':doc['user_id'],
                        'When Created:':doc["created_at"],
                       'Retweet Count':doc['retweet_count']
                        })

    return ret


In [ ]:
# frequently used function

"""
idk what's mean for sort('interactions')
"""
def get_tweet_by_ids(tweet_ids):
    tweet_data = tweets.find({'id':{'$in':tweet_ids}}).sort('interactions', -1)
    return tweets.find({'id':{'$in':tweet_ids}}).sort('interactions', -1)
   

def get_metadata_from_tweet(tweet_data):
    return tweet_data['user_id'],tweet_data['created_at'],len(tweet_data['retweets'])

In [ ]:
def get_tweet_id_by_user_name(sc_name):
    """ get tweet_ids values from corresponding user""" 
    sql =   """SELECT tweet_ids FROM user_db WHERE screen_name = %s"""
    try:
        # execute the UPDATE statement
        cur.execute("ROLLBACK")
        cur.execute(sql, (sc_name,))
        output = cur.fetchall()
        if len(output) !=0:
            return output[0][0]
    except (Exception, psycopg2.DatabaseError) as error:
        print(error) 


def search_by_user(user_name):
    tweet_ids = get_tweet_id_by_user_name(user_name)
    #want a get_user_name_by_user_id for meta data
    return get_tweet_by_ids(tweet_ids)

In [ ]:
def retweet_info(tweet_id):
    retweet_data = []
    tweet_data = tweets.find_one({"id": tweet_id})
    retweets = tweet_data['retweets']
    for retweet in retweets:
        retweet_info = {}
        retweet_info['User_id who retweeted this tweet'] = retweet['user_id']
       # screen_name = user_database.index[user_database['user_id']==retweet['user_id']].tolist()
       # retweet_info['screen_name'] = screen_name
        retweet_info['retweeted_at'] = retweet['created_at']
        retweet_data.append(retweet_info)
    print("Original Tweet " ,tweet_data['text'])
    print()
    print('The Retweet info about this tweet is following')
    return retweet_data


In [ ]:
def time_range_search():
    start_dt = ''
    print('Enter a start date of your search(in the format of YEAR-MONTH-DAY ex)2020-04-12): You can press enter to not set the start date.')
    start_date = input()
    if type(start_date) != str:
        start_date = start_date.toString()
    if start_date != '':
        print('Enter a start date time of your search (in the format of HOUR:MINUTE:SECOND ex)18:26:00):')
        start_time = input()
        if type(start_time) != str:``
            start_time = start_time.toString()
        start_dt = start_date + "T" + start_time   
        
    
    end_dt = ''
    print('Enter a end date of your search (in the format of YEAR-MONTH-DAY ex)2020-04-12): You can press enter to not set the start date.')
    end_date = input()
    if type(end_date) != str:
        end_date = end_date.toString()
    if end_date != '':
        print('Enter a end date time of of your search (in the format of HOUR:MINUTE:SECOND ex)18:26:00):')
        end_time = input()
        if type(end_time) != str:
            end_time = end_time.toString()
        end_dt = end_date + "T" + end_time  
        
    if start_dt != '' and end_dt != '':
        query={'created_at':{'$gt':start_dt,'$lt':end_dt}}
    elif start_dt != '':
        print('Searching from ' + start_dt)  
        query={'created_at':{'$gt':start_dt}}
    elif end_dt != '':
        print('Searching to ' + end_dt)  
        query={'created_at':{'$lt':end_dt}}
    else:
        query={}
        
    for i in tweets.data.find(query).sort('interactions', -1):
        print('NEW NEW')
        print('')
        print('')
        print(i)

    return tweets.data.find(query).sort('interactions', -1)


### Connecting to DB

In [ ]:
# connecting to the database
conn = psycopg2.connect(
    host = "localhost", #localhost
    port = "5432", #yourport
    database = "postgres", # databasename
    user="postgres",
    password =  my_pwd
)
cur = conn.cursor() # make  cursor 
client = pymongo.MongoClient("mongodb://localhost:27017/") 


# Create DB 
db = client["myDB"]
tweets=db["tweets_table"]
cur.execute("ROLLBACK")
cur.execute("CREATE TABLE user_db (screen_name VARCHAR (100) PRIMARY KEY, user_id bigint,followers_count int, tweet_ids bigint[]);")


# Create Index for MongoDB
tweets.create_index([('created_at', pymongo.DESCENDING)])
tweets.create_index([('user_id', pymongo.ASCENDING)])
tweets.create_index([('id', pymongo.DESCENDING)])

DuplicateTable: relation "user_db" already exists


In [ ]:
cur.execute("""SELECT * FROM user_db LIMIT 10;""")
cur.fetchall()

[('juwelz_v', 1242817830946508801, 43, [1249403767180668930]),
 ('Adakisn', 4707764075, 76, [1249403770435493888]),
 ('TVConservador', 988174833849634816, 7240, [1249403770360016896]),
 ('charlesmire', 270088915, 1295, [1249403770833981440]),
 ('Physics_Ashu', 2515708525, 25, [1249403771077005315]),
 ('meElfpunkt', 1171484224244744198, 3, [1249403771261722624]),
 ('abhishttripathi', 2341644176, 595, [1249403771676815361]),
 ('riaz_nov', 2950846039, 4, [1249403771781816323]),
 ('erinthomasx', 2905457446, 201, [1249403772679438340]),
 ('Brateki19662327', 1220761862452776965, 186, [1249403773178560515])]

In [ ]:
a=0
for x in tweets.find():
    print(x)
    a+=1
    if a == 10:
        break

{'_id': ObjectId('62673ef2a5bbd0c81b44e5f6'), 'id': 1249378751349231616, 'text': 'wishing death on people is weirdo behavior.', 'text_sorted': ['behavior', 'death', 'is', 'nuffsaidny', 'on', 'people', 'rt', 'weirdo', 'wishing'], 'created_at': '2020-04-12T16:48:01', 'user_id': 16144221, 'favorite_count': 50, 'retweet_count': 24, 'interactions': 74, 'hashtags': [], 'retweets': [{'id': 1249403767180668930, 'created_at': '2020-04-12T18:27:25', 'user_id': 1242817830946508801}]}
{'_id': ObjectId('62673ef2a5bbd0c81b44e5f7'), 'id': 1249397541596286979, 'text': 'In Turkey, there are 300 thousand prisoners and 150 thousand prison employees in prisons. \nPrisons are the most ris… https://t.co/GogsgfUxkE', 'text_sorted': ['150', '300', 'and', 'are', 'are', 'employees', 'in', 'in', 'lale_karanfil', 'most', 'pl', 'prison', 'prisoners', 'prisons', 'prisons', 'risky', 'rt', 'the', 'there', 'thousand', 'thousand', 'turkey'], 'created_at': '2020-04-12T18:02:41', 'user_id': 1087735689091928064, 'favorite

## Test for Search

#### Search By User

In [ ]:
%%time
res_user = search_by_user('juwelz_v')

CPU times: user 1.41 ms, sys: 1.32 ms, total: 2.73 ms
Wall time: 2.01 ms


#### Search By String

In [ ]:
%%time
search_by_text()

Enter a word you want to search:
covid
Searching for covid ...
CPU times: user 157 ms, sys: 11.4 ms, total: 169 ms
Wall time: 1.05 s


[{'Tweet Text': 'Someone should just tell him this is actualy Covid 16 he’ll be all over it',
  'Author of Tweet': 858716964,
  'When Created:': '2020-04-11T20:05:26',
  'Retweet Count': 522},
 {'Tweet Text': 'https://t.co/mA8joe6ut1\nDr Penyakit Dalam, Heo Yeong Gu meninggal ketularan pasien Covid 19 yg dirawatnya, Presiden… https://t.co/rjPYexbvhm',
  'Author of Tweet': 843793805010644992,
  'When Created:': '2020-04-11T10:41:35',
  'Retweet Count': 1164},
 {'Tweet Text': 'खुद्दार मेरे शहर का, आज भूख से मर गया.\n\nराशन तो लेना था, पर फोटो से डर गया..!\n\n#Corona  #Covid #Staysafe #karuna… https://t.co/8SLuO76Vv9',
  'Author of Tweet': 720101708,
  'When Created:': '2020-04-11T18:38:01',
  'Retweet Count': 331},
 {'Tweet Text': 'Tolong pemerintah sediakan lebih banyak lagi APD, bukan hanya utk RS khusus COVID.\n\nTetangga jarak 6 rumah dari say… https://t.co/RyhD8dVjqM',
  'Author of Tweet': 230696471,
  'When Created:': '2020-04-12T01:09:19',
  'Retweet Count': 452},
 {'Tweet Text': 

#### Search By HashTag

In [ ]:
%%time
search_by_ht()

Enter a word you want to search:
covid
Searching for covid ...
CPU times: user 138 ms, sys: 12 ms, total: 150 ms
Wall time: 1.12 s


[{'Tweet Text': 'खुद्दार मेरे शहर का, आज भूख से मर गया.\n\nराशन तो लेना था, पर फोटो से डर गया..!\n\n#Corona  #Covid #Staysafe #karuna… https://t.co/8SLuO76Vv9',
  'Author of Tweet': 720101708,
  'When Created:': '2020-04-11T18:38:01',
  'Retweet Count': 331},
 {'Tweet Text': 'The @EU_Commission takes initiatives to fund more #research on #COVID-19 &amp; supports participants to carry on with… https://t.co/HAa2va8LNB',
  'Author of Tweet': 2162942852,
  'When Created:': '2020-04-09T16:07:44',
  'Retweet Count': 112},
 {'Tweet Text': 'Its been18days of lockdown and the cases of corona is increasing daybyday.The fear and chaos of #COVID-19 all aroun… https://t.co/KW7UwpIuyb',
  'Author of Tweet': 3145274737,
  'When Created:': '2020-04-12T12:09:35',
  'Retweet Count': 17},
 {'Tweet Text': '#COVID-19\n#Lockdown\n#PoliceRequestToPublic\n\n"तोड़ देंगे तुम्हारे शरीर का,\nकोना कोना, \nलेकिन नही होने देंगें,\nतुमको… https://t.co/iuyEYPCidL',
  'Author of Tweet': 1224367896538730496,
  'When Cre

#### Search By Time Range

In [ ]:
### working on

#### Search By Who Retweeted

In [ ]:
retweet_info(1238612571193827335)

Original Tweet  If I gave you 100 skittles and told you 3 of them could kill you.... I’m sure you would avoid the fucking skittles

The Retweet info about this tweet is following


[{'User_id who retweeted this tweet': 183036721,
  'retweeted_at': '2020-04-12T18:43:16'},
 {'User_id who retweeted this tweet': 2858252304,
  'retweeted_at': '2020-04-12T18:44:19'},
 {'User_id who retweeted this tweet': 67095221,
  'retweeted_at': '2020-04-12T18:45:47'}]

In [ ]:
###
# If the user clicks on the author, 
# show other tweets by that author or retweets by that author (if available).

In [ ]:
### hashtags/users may be popular and their data may be cached.

In [ ]:
# When will the cache get updated? Can the data get stale? How will you purge stale data?

In [ ]:
def get_top10_user():
    top10_user = []
    cur.execute("""SELECT screen_name FROM user_db ORDER BY followers_count DESC LIMIT 10;""")
    res = cur.fetchall()
    for a in res:
        top10_user.append(a[0])
        
    return top10_user

def get_top10_tweets():
    res = []
    for doc in tweets.find().sort('interactions', -1)[:10]:
        tweet_info = {} 
        tweet_info['Tweet ID'] = doc['id']
        tweet_info['Tweet']  = doc['text']
        tweet_info['Popular Score'] = doc['interactions']
        res.append(tweet_info)
    return res

### Redis

### connection

In [ ]:
import redis
r = redis.StrictRedis(host='localhost', port=6379, db=0,charset="utf-8",
                      decode_responses=True)

#### Helper Function

In [ ]:
def get_top10_user():
    top10_user = {}
    cur.execute("""SELECT screen_name,followers_count FROM user_db ORDER BY followers_count DESC LIMIT 10;""")
    res = cur.fetchall()
    for a in res:
        top10_user[a[0]] =  int(a[1])
        
    return top10_user

def get_top10_tweets():
    res = {}
    for doc in tweets.find().sort('interactions', -1)[:10]:
        res[doc['text']] = doc['interactions']
    return res

###  1. Top Users

In [ ]:
def redis_store_top10_user(top10user):
    try:
        # delete before 
        r.delete('top10_user')
        tmp = top10user
        r.zadd('top10_user', tmp)
        print("Successfully Stored!")
    except:
        print("ERROR!")
        
def redis_get_top10_user():
    return r.zrange('top10_user',0, 10,-1) 
    
def redis_update_top10_user(top10user):
    # input top10user will be dictionary format ('screen_name':value)
    # top10user from the current user db
    # regis_get_top10_user form redis db
    if redis_get_top10_user() == list(top10user.keys()):
        # update the new value to the lateset user db
        r.delete('top10_user')
        redis_store_top10_user(top10user)
        print("Successfuly updated")
        

#### TEST FOR TOP USERS

In [ ]:
# store top 10 user to Redis
usertop10 = get_top10_user()
redis_store_top10_user(usertop10)

Successfully Stored!


In [ ]:
# get the data from redis
res = redis_get_top10_user()
res 

['detikcom',
 'la_patilla',
 'skynewsarabia',
 'virsanghvi',
 'NTelevisa_com',
 'repubblica',
 'casspernyovest',
 'OKAZ_online',
 'andihiyat',
 'Foro_TV']

### 2. Top Tweets

In [ ]:
def redis_store_top10_tweets(top10tweet):
    # input top10tweets will be dictionary format ('screen_name':value)
    try:
        # delete before 
        r.delete('top10_tweets')
        r.zadd('top10_tweets', top10tweet)
        print("Successfully Stored!")
        return 
    except:
        print("ERROR!")
    
def redis_get_top10_tweets():
    return r.zrange('top10_tweets',0, 10,-1) 
    

def redis_update_top10_tweets(top10tweets):
    # input top10tweets will be dictionary format ('screen_name':value)
    # top10user from the current user db
    # regis_get_top10_user form redis db
    if redis_get_top10_tweets() == list(top10tweets.keys()):
        # update the new value to the lateset user db
        r.delete('top10_tweets')
        redis_store_top10_tweets(top10tweets)
        print("Successfuly updated")

In [ ]:
# store top 10 user to Redis
tweet10 = get_top10_tweets()
redis_store_top10_tweets(tweet10)

Successfully Stored!


In [ ]:
# get the data from redis
res = redis_get_top10_tweets()
res 

['ALERT‼️‼️‼️\nThe corona virus can be spread through money. If you have any money at home, put on some gloves, put al… https://t.co/juJjDpFN3I',
 'If I gave you 100 skittles and told you 3 of them could kill you.... I’m sure you would avoid the fucking skittles',
 'THIS MAN IS A GENIUS he figured out the Corona virus problem 😮 https://t.co/EZP7IqTtxV',
 'It wasn’t no corona till y’all started balancing brooms in the house, y’all let the devil in',
 'Watch this. It shows why we should all do the right thing and stay home to the fullest extent possible. All of us c… https://t.co/GOODRTNI2e',
 'we all know who the only person who can defeat corona is https://t.co/u7DraiQHHY',
 'This what Corona do when you outta school for 2 weeks &amp; home w ya mama 😂😂😂🤦🏾\u200d♀️ https://t.co/bz4u2p2JZM',
 'CDC just announced that corona can be spread via Instagram story challenges :-/',
 'Ms.Corona made me do it 🤍🗻 https://t.co/PMfVcXwdoi',
 'Corona virus....its coming']

## Copmaring the time with Redis and without Redis

#### TOP USERS

### 10 Times Faster!!!

In [ ]:
# without Redis for top10 users
%%time
for _ in range(10):
    get_top10_user()

CPU times: user 1.89 ms, sys: 2.53 ms, total: 4.41 ms
Wall time: 41.4 ms


In [ ]:
%%time
# With Redis for top10 users
for _ in range(10):
    redis_get_top10_user()

CPU times: user 3.68 ms, sys: 1.99 ms, total: 5.67 ms
Wall time: 4.76 ms


#### TOP TWEETS

### 16 Times Faster!!!

In [ ]:
%%time
#without Redis for top10 tweets
for _ in range(10):
    get_top10_tweets()

CPU times: user 11.3 ms, sys: 2.03 ms, total: 13.3 ms
Wall time: 83.4 ms


In [ ]:
%%time
# With Redis for top10 tweets
for _ in range(10):
    redis_get_top10_tweets()

CPU times: user 4.86 ms, sys: 3.14 ms, total: 8 ms
Wall time: 5.99 ms
